# Memory Management - Part 1 - Sessions

In this notebook, you'll learn:

- ✅ What sessions are and how to use them in your agent
- ✅ How to build *stateful* agents with sessions and events
- ✅ How to persist sessions in a database
- ✅ Context management practices such as context compaction
- ✅ Best practices for sharing session State

---
# 1. Setup

In [1]:
import os
from dotenv import load_dotenv 

load_dotenv()
try:
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

✅ Gemini API key setup complete.
✅ ADK components imported successfully.


### Helper functions

Helper function that manages a complete conversation session, handling session
creation/retrieval, query processing, and response streaming. It supports
both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```

In [2]:
# Define helper functions that will be reused throughout the notebook

async def run_session(
    runner_instance: Runner,
    # XZ: pass explicitly the two arguments
    #--------------------------------------
    session_service: DatabaseSessionService,
    user_id: str,
    model_name: str,
    # -------------------------------------
    user_queries: list[str] | str = None,
    session_name: str = "default",

):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=user_id, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=user_id, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if isinstance(user_queries,str):
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print('-'*100)
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            message = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=user_id, session_id=session.id, new_message=message
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{model_name} > ", event.content.parts[0].text)
    else:
        print("No queries!")

print("✅ Helper functions defined.")

✅ Helper functions defined.


---
# 2. Session Management

**The Problem**

LLMs are stateless—they know only what you provide in a single API call. Without context management, every prompt starts from scratch.
ADK uses Sessions for short-term memory and Memory for long-term. This notebook covers Sessions.

**What is a Session**

A Session stores one continuous conversation: its history, tool calls, and responses. Each session belongs to one user and one agent.

Two components:
- Events: the conversation's building blocks—user inputs, agent responses, tool calls, tool outputs.
- State: a {key: value} scratchpad accessible to all subagents and tools.


**Managing Sessions**
- SessionService: stores and retrieves sessions (implementations vary: in-memory, database, cloud).
- Runner: orchestrates the conversation flow and maintains history automatically.

### Implementing First Stateful Agent

Lets start with a simple Session Management option (`InMemorySessionService`):

In [3]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"


# Step 1: Create the LLM Agent
root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Description of the agent's purpose
)

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


In [5]:
# Run a conversation with two queries in the same session
# Context is maintained
await run_session(
    runner,
    session_service,
    USER_ID,
    MODEL_NAME,
    [
        "Hello! who is P li",
        "Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生",
        "who is p li",
        "虎俊生 who"
    ],
    "first_p_li_session",
)


 ### Session: first_p_li_session
----------------------------------------------------------------------------------------------------

User > Hello! who is P li
gemini-2.5-flash-lite >  I'm sorry, but I don't have enough information to answer your question. "P li" could refer to many different people. Could you please provide more context or details? For example, are you thinking of:

*   A specific person in history?
*   A character from a book, movie, or TV show?
*   A public figure you've heard about?
*   Someone you know personally?

The more information you can give me, the better I can help!
----------------------------------------------------------------------------------------------------

User > Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生
gemini-2.5-flash-lite >  Oh, thank you for the clarification! That's wonderful to know. So, P Li is a very special labradoodle, and you consider her the cutest of all! And her Chinese n

🎉 **Success!** The agent remembered your name because both queries were part of the same session. The Runner automatically maintained the conversation history.

But there's a catch: `InMemorySessionService` is temporary. **Once the application stops, all conversation history is lost.** 


In [6]:
# Run this cell after restarting the kernel WITHOUT running the previous . All this history will be gone...
await run_session(
    runner,
    session_service,
    USER_ID,
    MODEL_NAME,
    [
        "Hello! what did we talk about",
    ],
    "first_p_li_session",
)


 ### Session: first_p_li_session
----------------------------------------------------------------------------------------------------

User > Hello! what did we talk about
gemini-2.5-flash-lite >  We've been talking about **P Li**, who you've described as:

*   A **labradoodle**.
*   The **cutest of all**.
*   Having the Chinese name **虎俊生 (Hǔ Jùn Shēng)**.


---
# 3. Persistent Sessions with `DatabaseSessionService`

While `InMemorySessionService` is great for prototyping, real-world applications need conversations to survive restarts, crashes, and deployments. Let's level up to persistent storage!

### Choosing the Right SessionService*

ADK provides different SessionService implementations for different needs:

| Service | Use Case | Persistence | Best For |
|---------|----------|-------------|----------|
| **InMemorySessionService** | Development & Testing | ❌ Lost on restart | Quick prototypes |
| **DatabaseSessionService** | Self-managed apps | ✅ Survives restarts | Small to medium apps |
| **Agent Engine Sessions** | Production on GCP | ✅ Fully managed | Enterprise scale |


Let's upgrade to one persistent session service type: `DatabaseSessionService` using SQLite.

In [7]:
# Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
# !!! modified here
db_url = "sqlite+aiosqlite:///my_agent_data.db" # "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [6]:
await run_session(
    runner,
    session_service,
    USER_ID, 
    MODEL_NAME,
    [   "i am sam",
        "Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生",
        "who is p li",
        "虎俊生 who"],
    "first_p_li_session_with_db",
)


 ### Session: first_p_li_session_with_db
----------------------------------------------------------------------------------------------------

User > i am sam
gemini-2.5-flash-lite >  Hello Sam, it's nice to meet you! I'm a text chatbot. How can I help you today?
----------------------------------------------------------------------------------------------------

User > Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生
gemini-2.5-flash-lite >  Okay, Sam! I will remember that:

*   **Pli** is a labradoodle.
*   Pli is the cutest of all.
*   Pli's Chinese name is 虎俊生 (Hǔ Jùn Shēng).
----------------------------------------------------------------------------------------------------

User > who is p li
gemini-2.5-flash-lite >  Pli is a labradoodle, and you've mentioned she's the cutest of all! Her Chinese name is 虎俊生.
----------------------------------------------------------------------------------------------------

User > 虎俊生 who
gemin

In [8]:
# Run this cell after restarting the kernel WITHOUT running the previous . All this history is persistent

await run_session(
    runner,
    session_service,
    USER_ID, 
    MODEL_NAME,
    [   "what did i tell you about p li"],
    "first_p_li_session_with_db",
)


 ### Session: first_p_li_session_with_db
----------------------------------------------------------------------------------------------------

User > what did i tell you about p li
gemini-2.5-flash-lite >  You told me that Pli is a labradoodle who is the cutest of all, and her Chinese name is 虎俊生.


In [9]:
# verify that the session data is isolated BY creating a new session
await run_session(
    runner, session_service, USER_ID, MODEL_NAME, ["Hello! Who is p li?"], "a_random_session"
) 


 ### Session: a_random_session
----------------------------------------------------------------------------------------------------

User > Hello! Who is p li?
gemini-2.5-flash-lite >  I do not have any information about "p li" in my current knowledge base. Can you provide more context or details about this person? For example, their full name, profession, or any specific achievements they are known for?


In [10]:
# check how data (events) is stored
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, timestamp, event_data from events"
        )

        cols_list = [_[0] for _ in result.description]
        print(cols_list)

        rows = []
        for each in result.fetchall():
            rows.append(each)

        return cols_list, rows


cols, rows = check_data_in_db()

import pandas as pd 
pd.set_option('display.max_colwidth', None)
pd.DataFrame(rows, columns=cols)

['app_name', 'session_id', 'timestamp', 'event_data']


,app_name,session_id,timestamp,event_data
0,default,first_p_li_session_with_db,2026-01-31 20:17:03.679620,"{""content"": {""parts"": [{""text"": ""i am sam""}], ""role"": ""user""}, ""invocation_id"": ""e-ec4d2c7b-a190-4909-a6e6-53d0e6a4c3f9"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""d3b813b6-94b4-4e61-b539-af013e09c5c6"", ""timestamp"": 1769919423.67962}"
1,default,first_p_li_session_with_db,2026-01-31 20:17:03.683956,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""Hello Sam, it's nice to meet you! I'm a text chatbot. How can I help you today?""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 25, ""prompt_token_count"": 34, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 34}], ""total_token_count"": 59}, ""invocation_id"": ""e-ec4d2c7b-a190-4909-a6e6-53d0e6a4c3f9"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""acb51de9-a3b6-4e8f-930a-f667434cc283"", ""timestamp"": 1769919423.683956}"
2,default,first_p_li_session_with_db,2026-01-31 20:17:04.255913,"{""content"": {""parts"": [{""text"": ""Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name \u864e\u4fca\u751f""}], ""role"": ""user""}, ""invocation_id"": ""e-3f6b65d0-688a-4118-ba33-5c1a9a1fbdf3"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""47222b43-e7ac-4685-bb4d-f28f8f32d954"", ""timestamp"": 1769919424.255913}"
3,default,first_p_li_session_with_db,2026-01-31 20:17:04.259644,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""Okay, Sam! I will remember that:\n\n* **Pli** is a labradoodle.\n* Pli is the cutest of all.\n* Pli's Chinese name is \u864e\u4fca\u751f (H\u01d4 J\u00f9n Sh\u0113ng).""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 56, ""prompt_token_count"": 89, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 89}], ""total_token_count"": 145}, ""invocation_id"": ""e-3f6b65d0-688a-4118-ba33-5c1a9a1fbdf3"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""f212e44a-826b-4b8f-a6ad-22c0754b7465"", ""timestamp"": 1769919424.259644}"
4,default,first_p_li_session_with_db,2026-01-31 20:17:04.865081,"{""content"": {""parts"": [{""text"": ""who is p li""}], ""role"": ""user""}, ""invocation_id"": ""e-8a51e578-4b87-4acf-b8a7-b5639f1c0bbb"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""fe075eee-04cf-4126-b6c4-4a240fdc05a8"", ""timestamp"": 1769919424.865081}"
5,default,first_p_li_session_with_db,2026-01-31 20:17:04.867952,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""Pli is a labradoodle, and you've mentioned she's the cutest of all! Her Chinese name is \u864e\u4fca\u751f.""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 30, ""prompt_token_count"": 151, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 151}], ""total_token_count"": 181}, ""invocation_id"": ""e-8a51e578-4b87-4acf-b8a7-b5639f1c0bbb"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""11d7aed6-8c7f-4b1c-8374-4d9ac9d5c509"", ""timestamp"": 1769919424.867952}"
6,default,first_p_li_session_with_db,2026-01-31 20:17:05.488192,"{"

---# 4. Context CompactionStoring all events in full makes sessions grow quickly. Long conversations become slow and expensive.**Context Compaction** automatically summarizes past events to reduce context size.

### 4.1 Create an App for the agent

To enable this feature, let's use the same `chatbot_agent` we created in Section 3.2. 

The first step is to create an object called `App`. We'll give it a name and pass in our chatbot_agent. 

We'll also create a new config to do the Context Compaction. This **`EventsCompactionConfig`** defines two key variables:

- **compaction_interval**: Asks the Runner to compact the history after every `n` conversations
- **overlap_size**: Defines the number of previous conversations to retain for overlap

We'll then provide this app to the Runner.


In [19]:
# Re-define our app with Events Compaction enabled
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite+aiosqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅ Research App upgraded with Events Compaction!")

✅ Research App upgraded with Events Compaction!


/var/folders/c1/gdb4b0xx4vd70kccs_c2tq3c0000gn/T/ipykernel_99307/1882568348.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


### 4.2 Running the Demo

Now, let's have a conversation that is long enough to trigger the compaction. When you run the cell below, the output will look like a normal conversation. However, because we configured our `App`, a compaction process will run silently in the background after the 3rd invocation.

In the next step, we'll prove that it happened.

In [20]:
# Turn 1
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)

# Turn 2
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)

# Turn 4
await run_session(
    research_runner_compacting,
    "Who are the main companies involved in that?",
    "compaction_demo",
)


 ### Session: compaction_demo

User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  Here's a summary of some of the latest news and trends in AI in healthcare:

**Key Areas of Advancement and Focus:**

*   **Drug Discovery and Development:** AI continues to be a powerhouse in accelerating drug discovery. Companies are using AI to identify novel drug targets, predict the efficacy and toxicity of potential drug candidates, and design new molecules. This is leading to faster and more cost-effective development pipelines.
    *   **Recent Developments:** Look for news about AI platforms that are identifying new uses for existing drugs (drug repurposing) or designing entirely new therapeutic compounds.
*   **Diagnostic Imaging:** AI algorithms are becoming increasingly sophisticated at analyzing medical images (X-rays, CT scans, MRIs, pathology slides) to detect diseases like cancer, diabetic retinopathy, and neurological disorders earlier and with greater accura

### 4.3 Verifying Compaction in the Session History

The conversation above looks normal, but the history has been changed behind the scenes. How can we prove it?

We can inspect the `events` list from our session. The compaction process **doesn't delete old events; it replaces them with a single, new `Event` that contains the summary.** Let's find it.

In [22]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

✅ SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None interaction_id=None invocation_id='c7ffa564-43fb-4104-be70-df0ef7783228' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=EventCompaction(start_timestamp=1769908120.203446, end_timestamp=1769908129.826301, compacted_content=Content(
  parts=[
    Part(
      text="""The user inquired about the latest news in AI in healthcare. The AI agent provide

### 4.4 What you've accomplished: Automatic Context Management

You just found the proof! The presence of that special summary `Event` in your session's history is the tangible result of the compaction process.

**Let's recap what you just witnessed:**

1.  **Silent Operation**: You ran a standard conversation, and from the outside, nothing seemed different.
2.  **Background Compaction**: Because you configured the `App` with `EventsCompactionConfig`, the ADK `Runner` automatically monitored the conversation length. Once the threshold was met, it triggered the summarization process in the background.
3.  **Verified Result**: By inspecting the session's events, you found the summary that the LLM generated. This summary now replaces the older, more verbose turns in the agent's active context.

**For all future turns in this conversation, the agent will be given this concise summary instead of the full history.** This saves costs, improves performance, and helps the agent stay focused on what's most important.


### 4.5 More Context Engineering options in ADK

#### 👉 Custom Compaction
In this example, we used ADK's default summarizer. For more advanced use cases, you can provide your own by defining a custom `SlidingWindowCompactor` and passing it to the config. This allows you to control the summarization prompt or even use a different, specialized LLM for the task. You can read more about it in the [official documentation](https://google.github.io/adk-docs/context/compaction/).

#### 👉 Context Caching
ADK also provides **Context Caching** to help reduce the token size of the static instructions that are fed to the LLM by caching the request data. Read more about it [here](https://google.github.io/adk-docs/context/caching/).

### The Problem

While we can do Context Compaction and use a database to resume a session, we face new challenges now. In some cases, **we have key information or preferences that we want to share across other sessions.** 

In these scenarios, instead of sharing the entire session history, transferring information from a few key variables can improve the session experience. Let's see how to do it!

---
## 🤝 Section 5: Working with Session State

### 5.1 Creating custom tools for Session state management

Let's explore how to manually manage session state through custom tools. In this example, we'll identify a **transferable characteristic**, like a user's name and their country, and create tools to capture and save it.

**Why This Example?**

The username is a perfect example of information that:

- Is introduced once but referenced multiple times
- Should persist throughout a conversation
- Represents a user-specific characteristic that enhances personalization

Here, for demo purposes, we'll create two tools that can store and retrieve user name and country from the Session State. **Note that all tools have access to the `ToolContext` object.** You don't have to create separate tools for each piece of information you want to share. 

In [23]:
# Define scope levels for state keys (following best practices)
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# This demonstrates how tools can write to session state using tool_context.
# The 'user:' prefix indicates this is user-specific data.
def save_userinfo(
    tool_context: ToolContext, user_name: str, country: str
) -> Dict[str, Any]:
    """
    Tool to record and save user name and country in session state.

    Args:
        user_name: The username to store in session state
        country: The name of the user's country
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}


# This demonstrates how tools can read from session state.
def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve user name and country from session state.
    """
    # Read from session state
    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")

    return {"status": "success", "user_name": user_name, "country": country}


print("✅ Tools created.")

✅ Tools created.


**Key Concepts:**
- Tools can access `tool_context.state` to read/write session state
- Use descriptive key prefixes (`user:`, `app:`, `temp:`) for organization
- State persists across conversation turns within the same session

### 5.2 Creating an Agent with Session State Tools

Now let's create a new agent that has access to our session state management tools:

In [24]:
# Configuration
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Create an agent with session state tools
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="""A text chatbot.
    Tools for managing user context:
    * To record username and country when provided use `save_userinfo` tool. 
    * To fetch username and country when required use `retrieve_userinfo` tool.
    """,
    tools=[save_userinfo, retrieve_userinfo],  # Provide the tools to the agent
)

# Set up session service and runner
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")

print("✅ Agent with session state tools initialized!")

✅ Agent with session state tools initialized!


### 5.3 Testing Session State in Action

Let's test how the agent uses session state to remember information across conversation turns:

In [25]:
# Test conversation demonstrating session state
await run_session(
    runner,
    [
        "Hi there, how are you doing today? What is my name?",  # Agent shouldn't know the name yet
        "My name is Sam. I'm from Poland.",  # Provide name - agent should save it
        "What is my name? Which country am I from?",  # Agent should recall from session state
    ],
    "state-demo-session",
)


 ### Session: state-demo-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing well, thank you for asking. I don't have access to your name. Can you please tell me what it is? 


User > My name is Sam. I'm from Poland.


/Users/xing.zhang/anaconda3/envs/google-adk/lib/python3.11/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


gemini-2.5-flash-lite >  I've saved your name as Sam and your country as Poland. It's nice to meet you!

User > What is my name? Which country am I from?
gemini-2.5-flash-lite >  Your name is Sam and you are from Poland.


### 5.4 Inspecting Session State

Let's directly inspect the session state to see what's stored:

In [26]:
# Retrieve the session and inspect its state
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session"
)

print("Session State Contents:")
print(session.state)
print("\n🔍 Notice the 'user:name' and 'user:country' keys storing our data!")

Session State Contents:
{'user:name': 'Sam', 'user:country': 'Poland'}

🔍 Notice the 'user:name' and 'user:country' keys storing our data!


### 5.5 Session State Isolation

As we've already seen, an important characteristic of session state is that it's isolated per session. Let's demonstrate this by starting a new session:

In [27]:
# Start a completely new session - the agent won't know our name
await run_session(
    runner,
    ["Hi there, how are you doing today? What is my name?"],
    "new-isolated-session",
)

# Expected: The agent won't know the name because this is a different session


 ### Session: new-isolated-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing great, thank you for asking. I'm not sure what your name is, though. Would you like to tell me? 



### 5.6 Cross-Session State Sharing

While sessions are isolated by default, you might notice something interesting. Let's check the state of our new session (`new-isolated-session`):

In [28]:
# Check the state of the new session
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="new-isolated-session"
)

print("New Session State:")
print(session.state)

# Note: Depending on implementation, you might see shared state here.
# This is where the distinction between session-specific and user-specific state becomes important.

New Session State:
{'user:name': 'Sam', 'user:country': 'Poland'}


---

## 🧹 Cleanup

In [29]:
# Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")

✅ Cleaned up old database files


---
## 📊 Summary

🎉 Congratulations! You've learned the fundamentals of building stateful AI agents:

- ✅ **Context Engineering** - You understand how to assemble context for LLMs using Context Compaction
- ✅ **Sessions & Events** - You can maintain conversation history across multiple turns
- ✅ **Persistent Storage** - You know how to make conversations survive restarts
- ✅ **Session State** - You can track structured data during conversations
- ✅ **Manual State Management** - You've experienced both the power and limitations of manual approaches
- ✅ **Production Considerations** - You're ready to handle real-world challenges


---

## ✅ Congratulations! You did it 🎉

**ℹ️ Note: No submission required!**

This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

### 📚 Learn More

Refer to the following documentation to learn more:

- [ADK Documentation](https://google.github.io/adk-docs/)
- [ADK Sessions](https://google.github.io/adk-docs/)
- [ADK Session-State](https://medium.com/google-cloud/2-minute-adk-manage-context-efficiently-with-artifacts-6fcc6683d274)
- [ADK Session Compaction](https://google.github.io/adk-docs/context/compaction/#define-compactor)

### 🎯 Next Steps - Long Term Memory Systems (Part 2)

#### Why do we need memory?
In this notebook, we manually identified a couple characteristic (username and country) and built tools to manage it. But real conversations involve hundreds of such characteristics:
- User preferences and habits
- Past interactions and their outcomes
- Domain knowledge and expertise levels
- Communication styles and patterns
- Contextual relationships between topics

**The Memory System in ADK automates this entire process**, making it a valuable asset for building truly Context-Aware Agents that can accommodate any user's current and future needs.

In the next notebook (Part 2: Memory Management), you'll learn how to:
- Enable automatic memory extraction from conversations
- Build agents that learn and adapt over time
- Create truly personalized experiences at scale
- Manage long-term knowledge across sessions

Ready to transform your manual state management into an intelligent, automated Memory system? Let's continue to Part 2!

---

| Authors |
| --- |
| [Sampath M](https://www.linkedin.com/in/msampathkumar/) |